In [2]:
!pip install transformers

     |████████████████████████████████| 2.5MB 30.6MB/s 
     |████████████████████████████████| 3.3MB 32.9MB/s 
     |████████████████████████████████| 901kB 42.0MB/s 


In [1]:
import torch

In [3]:
!pip install datasets

     |████████████████████████████████| 266kB 31.3MB/s 
     |████████████████████████████████| 245kB 34.6MB/s 
     |████████████████████████████████| 122kB 45.1MB/s 


In [4]:
!pip install scikit-learn

In [5]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_metric

In [31]:
dataset = load_dataset('csv', data_files=['twitter15_text_TF.csv'])['train'].train_test_split(test_size=0.25)

Using custom data configuration default-c6f19c8dfe670bf4


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c6f19c8dfe670bf4/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [32]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7

In [33]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def tokenize(batch):
  tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=128)
  tokens['labels'] = labels.str2int(batch['labels'])
  return tokens

In [34]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [35]:
training_set = tokenized_datasets['train']#.select(range(100))
test_set = tokenized_datasets['test']#.select(range(100))


In [36]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4, )

training_args = TrainingArguments("./test_trainer", evaluation_strategy="epoch", num_train_epochs=10.0)


metric = load_metric("accuracy")


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab

In [37]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)

In [38]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running training *****
  Num examples = 556
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 700


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.306264,0.897849
2,No log,0.243854,0.935484
3,No log,0.211469,0.946237
4,No log,0.518904,0.913978
5,No log,0.366374,0.946237
6,No log,0.474589,0.919355
7,No log,0.420350,0.940860
8,0.116900,0.411988,0.940860
9,0.116900,0.412270,0.940860
10,0.116900,0.412400,0.940860


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequ

TrainOutput(global_step=700, training_loss=0.08359954192170076, metrics={'train_runtime': 646.837, 'train_samples_per_second': 8.596, 'train_steps_per_second': 1.082, 'total_flos': 1870044232581120.0, 'train_loss': 0.08359954192170076, 'epoch': 10.0})

In [39]:
metrics=trainer.evaluate()
print(metrics)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8


{'eval_loss': 0.4123995304107666, 'eval_accuracy': 0.9408602150537635, 'eval_runtime': 7.0974, 'eval_samples_per_second': 26.207, 'eval_steps_per_second': 3.382, 'epoch': 10.0}
